In [6]:
import os
import pandas as pd
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import copy
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import joblib
import optuna
from tqdm import tqdm
import time
import codecs

# Configuration

In [ ]:
rawdata_dir = 'rawdata'
data_dir = 'data'

lags = [1,2,3,6,12,24,36]

# NUM_FOLD_DAYS = 28
NUM_FOLD_DAYS = 56
VALID_FIRST_DAY = 1914
VALID_LAST_DAY = 1941
NUM_USE_DAY = 365
# TRAIN_FIRST_DAY = max(lags)
TRAIN_FIRST_DAY = VALID_FIRST_DAY - NUM_USE_DAY + 1

OPTUNA_TRIAL = 70

# Read data

In [8]:
sales = pd.read_csv(os.path.join(rawdata_dir, 'sales_train_evaluation.csv'))
sales.name = 'sales'
calendar = pd.read_csv(os.path.join(rawdata_dir, 'calendar.csv'))
calendar.name = 'calendar'
prices = pd.read_csv(os.path.join(rawdata_dir, 'sell_prices.csv'))
prices.name = 'prices'

Since, the validation data is now available for the days 1914-1941, Adding zero sales for days: d_1942 - d_1969(Test)

In [9]:
#Add zero sales for the remaining days 1942-1969
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

# Downcast

In [10]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

# Melt data

In [11]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

## Combine data
Combine price data from prices dataframe and days data from calendar dataset.

In [12]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left')

# Feature Engineering

## Label Encoding

In [13]:
#Store the categories along with their codes
d_id = dict(zip(df.id.cat.codes, df.id))
d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

In [14]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes
        
df.drop('date',axis=1,inplace=True)

## Introduce lags

In [15]:
#Introduce lags
def make_lags(df, lags):
    for lag in lags:
        df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)
        
    return df

In [16]:
df = make_lags(df, lags)

## make new features

In [17]:
def make_new_feature(df, d_last = df['d'].max()):
    # copy sold
    sold_tmp = copy.copy(df['sold'])
    
    # embed NaN
    df['sold'][df['d'] > d_last] = np.nan
    
    # make feature using sold
    df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
    df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
    df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
    df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
    df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
    df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
    df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    
    df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
    df['expanding_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)
    
    df['daily_avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])['sold'].transform('mean').astype(np.float16)
    df['avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform('mean').astype(np.float16)
    df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
    df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)
    
    # restore sold
    df['sold'] = sold_tmp
    
    return df

In [20]:
df_all = copy.copy(df)
df_all = make_new_feature(df_all)
df_all = df_all[df_all['d'] >= max(lags)]
df_all.info()

df_all.to_pickle(os.path.join(data_dir, 'data_all.pkl'))   # save df_all (use this after CV)
del df_all
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12805800 entries, 47229010 to 60034809
Data columns (total 43 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id                        int16  
 1   item_id                   int16  
 2   dept_id                   int8   
 3   cat_id                    int8   
 4   store_id                  int8   
 5   state_id                  int8   
 6   d                         int16  
 7   sold                      int16  
 8   wm_yr_wk                  int16  
 9   weekday                   int8   
 10  wday                      int8   
 11  month                     int8   
 12  year                      int16  
 13  event_name_1              int8   
 14  event_type_1              int8   
 15  event_name_2              int8   
 16  event_type_2              int8   
 17  snap_CA                   int8   
 18  snap_TX                   int8   
 19  snap_WI                   int8   
 20  sell_price     

0

# Cross Validation

## Cross Validation for each store (function)

In [37]:
def xgboost_cv_store(df, store):
    def objective(trial):
        # param space
        param = {
            'objective': 'reg:squarederror',
            'n_estimators': 1000,
            'learning_rate': 0.1,
            'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.95, 0.05),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 0.95, 0.05),
            'max_depth':  trial.suggest_int('max_depth', 3, 9),
            'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 0.1, 10.0),
            'tree_method': 'gpu_hist'
        }
        
        ### START: CV Loop ###
        isBreakCV = False
        fold_iter = 0
        rmse_mean = 0.
        while not(isBreakCV):
            start = time.time()   # start timer
            
            d_valid_last = VALID_LAST_DAY - NUM_FOLD_DAYS * fold_iter
            d_train_last = VALID_LAST_DAY - NUM_FOLD_DAYS * (fold_iter + 1)
            if d_train_last < TRAIN_FIRST_DAY + NUM_FOLD_DAYS * 2:
                isBreakCV = True
            
            # make_new_feature
            df_fold_file = os.path.join(data_dir, 'data_foldday'+str(NUM_FOLD_DAYS)+'_fold'+str(fold_iter + 1)+'.pkl')
            if os.path.isfile(df_fold_file):   # make_new_feature 適用後のファイルがあればそれを使う
                df_fold = pd.read_pickle(df_fold_file)
            else:
                df_fold = df[df['d'] <= d_valid_last]
                df_fold = make_new_feature(df_fold, d_train_last)   # train の情報だけを使って特徴量を作成
                df_fold.to_pickle(df_fold_file)   # pickleとして保存しておく

            # Extract store
            df_fold = df_fold[df_fold['store_id'] == store]
                        
            # Split data
            X_train = df_fold[(df_fold['d'] >= TRAIN_FIRST_DAY) & (df_fold['d'] <= d_train_last)].drop('sold', axis=1)
            y_train = df_fold[(df_fold['d'] >= TRAIN_FIRST_DAY) & (df_fold['d'] <= d_train_last)]['sold']
            X_valid = df_fold[(df_fold['d'] > d_train_last) & (df_fold['d'] <= d_valid_last)].drop('sold', axis=1)
            y_valid = df_fold[(df_fold['d'] > d_train_last) & (df_fold['d'] <= d_valid_last)]['sold']
            
            # Train XGBoost model
            model = xgb.XGBRegressor(**param)
#             model.fit(X_train, y_train, eval_set=[(X_train,y_train), (X_valid, y_valid)],
#                  eval_metric='rmse', verbose=10, early_stopping_rounds=100)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  eval_metric='rmse', verbose=10, early_stopping_rounds=50)
            y_pred = model.predict(X_valid)
            rmse_mean += np.sqrt(mean_squared_error(y_valid.values, y_pred))
            
            fold_iter += 1
            elapsed_time = time.time() - start   # stop timer
            print("[{0}] Fold{1}: {2} [sec]".format(d_store_id[store], fold_iter, elapsed_time))
        ### END: CV Loop ###
        
        del df_fold, model, X_train, y_train, X_valid, y_valid, y_pred
        gc.collect()
        
        rmse_mean /= fold_iter
        return rmse_mean
    return objective


## Execute CV

In [41]:
stores = sales.store_id.cat.codes.unique().tolist()

valid_preds = df[(df['d'] >= VALID_FIRST_DAY) & (df['d'] <= VALID_LAST_DAY)]['sold']
eval_preds = df[df['d'] > VALID_LAST_DAY]['sold']

# find best params by doing CV (optuna)
for store in tqdm(stores):
    print('##################### CV START: {0} #####################'.format(d_store_id[store]))
    study = optuna.create_study()
    study.optimize(xgboost_cv_store(df, store), n_trials=OPTUNA_TRIAL)
    
    # save optuna log
    df_trial = study.trials_dataframe()
    df_trial.to_pickle(os.path.join('report', 'OptunaTrials_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'))
    del df_trial
    gc.collect()
    
    # savae best params found by CV
    bestparam_filename = 'BestParamsSelectedByCV_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'wb') as f:
        pickle.dump(study.best_params, f)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

##################### CV START: CA_1 #####################
[0]	validation_0-rmse:3.76657
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:9.97533
[20]	validation_0-rmse:22.20261
[30]	validation_0-rmse:25.89632
[40]	validation_0-rmse:28.56722
[50]	validation_0-rmse:32.69363
Stopping. Best iteration:
[4]	validation_0-rmse:3.46721

[0]	validation_0-rmse:3.62713
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:10.98892
[20]	validation_0-rmse:14.80719
[30]	validation_0-rmse:17.40102
[40]	validation_0-rmse:21.07243
[50]	validation_0-rmse:25.10614
Stopping. Best iteration:
[2]	validation_0-rmse:3.59517

[0]	validation_0-rmse:3.57664
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:20.40320
[20]	validation_0-rmse:23.96271
[30]	validation_0-rmse:33.30601
[40]	validation_0-rmse:36.04877
[50]	validation_0-rmse:37.59591
Stopping. Best iteration:
[3]	validation_0-rmse:3.31062

[0]	va

[I 2020-06-24 01:31:42,832] Finished trial#0 with value: 3.5881975293159485 with parameters: {'subsample': 0.75, 'colsample_bytree': 0.95, 'max_depth': 9, 'gamma': 0.00010965888152054486, 'min_child_weight': 7.0778854578679455}. Best is trial#0 with value: 3.5881975293159485.


[0]	validation_0-rmse:3.75970
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:9.94763
[20]	validation_0-rmse:14.38096
[30]	validation_0-rmse:15.95799
[40]	validation_0-rmse:18.15580
[50]	validation_0-rmse:18.92608
Stopping. Best iteration:
[3]	validation_0-rmse:3.71665

[0]	validation_0-rmse:3.62036
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:6.92829
[20]	validation_0-rmse:11.95214
[30]	validation_0-rmse:19.56136
[40]	validation_0-rmse:21.87501
[50]	validation_0-rmse:23.76128
Stopping. Best iteration:
[2]	validation_0-rmse:3.60449

[0]	validation_0-rmse:3.57597
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:12.66184
[20]	validation_0-rmse:14.16053
[30]	validation_0-rmse:13.81879
[40]	validation_0-rmse:15.10357
[50]	validation_0-rmse:18.27483
Stopping. Best iteration:
[0]	validation_0-rmse:3.57597

[0]	validation_0-rmse:3.61755
Will train until validation_0-rmse h

[I 2020-06-24 01:37:26,404] Finished trial#1 with value: 3.588985025882721 with parameters: {'subsample': 0.85, 'colsample_bytree': 0.95, 'max_depth': 7, 'gamma': 0.00010563641448624307, 'min_child_weight': 7.271655995510915}. Best is trial#0 with value: 3.5881975293159485.


...

[0]	validation_0-rmse:3.67819
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:5.02196
[20]	validation_0-rmse:5.28009
[30]	validation_0-rmse:8.68721
[40]	validation_0-rmse:11.32860
[50]	validation_0-rmse:11.28762
Stopping. Best iteration:
[3]	validation_0-rmse:3.45563

[0]	validation_0-rmse:3.54386
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:4.79057
[20]	validation_0-rmse:13.60834
[30]	validation_0-rmse:18.33601
[40]	validation_0-rmse:20.96463
[50]	validation_0-rmse:21.98383
Stopping. Best iteration:
[3]	validation_0-rmse:3.31807

[0]	validation_0-rmse:3.49251
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:11.97411
[20]	validation_0-rmse:13.26825
[30]	validation_0-rmse:17.08254
[40]	validation_0-rmse:17.15999
[50]	validation_0-rmse:17.12042
Stopping. Best iteration:
[3]	validation_0-rmse:3.11892

[0]	validation_0-rmse:3.53733
Will train until validation_0-rmse has

[I 2020-06-24 10:56:17,435] Finished trial#98 with value: 3.233902633190155 with parameters: {'subsample': 0.95, 'colsample_bytree': 0.6, 'max_depth': 7, 'gamma': 5.30698444994494e-05, 'min_child_weight': 0.718430081897794}. Best is trial#45 with value: 3.233902633190155.


[0]	validation_0-rmse:3.68106
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:3.13025
[20]	validation_0-rmse:3.25190
[30]	validation_0-rmse:3.26020
[40]	validation_0-rmse:3.71171
[50]	validation_0-rmse:3.76846
[60]	validation_0-rmse:4.80874
Stopping. Best iteration:
[14]	validation_0-rmse:3.03491

[0]	validation_0-rmse:3.55684
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:5.82500
[20]	validation_0-rmse:5.91635
[30]	validation_0-rmse:11.37670
[40]	validation_0-rmse:12.14559
[50]	validation_0-rmse:15.50055
Stopping. Best iteration:
[2]	validation_0-rmse:3.21179

[0]	validation_0-rmse:3.50004
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:3.04338
[20]	validation_0-rmse:3.09150
[30]	validation_0-rmse:3.26806
[40]	validation_0-rmse:6.72392
[50]	validation_0-rmse:8.17714
[60]	validation_0-rmse:11.45647
[70]	validation_0-rmse:13.73054
Stopping. Best iteration:
[25]	valida

[I 2020-06-24 11:02:13,206] Finished trial#99 with value: 3.2852317690849304 with parameters: {'subsample': 0.95, 'colsample_bytree': 0.6, 'max_depth': 8, 'gamma': 1.405634402244118e-05, 'min_child_weight': 0.7511136942516804}. Best is trial#45 with value: 3.233902633190155.
  0%|                                                                                        | 0/10 [10:02:18<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'train_result\\best_params_CA_1_last365.pkl'

In [ ]:
del df
gc.collect()

## Train & Predict by best model (function)

In [ ]:
def xgboost_by_bestparam_for_valid(df, store, best_params, valid_preds):
    # prepare data set
    df_store = df[df['store_id'] == store]
    X_train = df_store[df_store['d'] < VALID_FIRST_DAY].drop('sold', axis=1)
    y_train = df_store[df_store['d'] < VALID_FIRST_DAY]['sold']
    X_valid = df_store[(df_store['d'] >= VALID_FIRST_DAY) & (df_store['d'] <= VALID_LAST_DAY)].drop('sold', axis=1)
    
    # train XGBoost by using best_params
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_metric='rmse')
    
    # save model for validation
    modelfile = 'BestModel4valid_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    
    # predict for valid
    valid_preds[X_valid.index] = model.predict(X_valid)
    del df_store, X_train, y_train, X_valid, model
    gc.collect()
    

In [ ]:
def xgboost_by_bestparam_for_test(df, store, best_params, eval_preds):
    # prepare data set
    df_store = df[df['store_id'] == store]
    X_train = df_store[df_store['d'] <= VALID_LAST_DAY].drop('sold', axis=1)
    y_train = df_store[df_store['d'] <= VALID_LAST_DAY]['sold']
    X_test = df_store[df_store['d'] > VALID_LAST_DAY].drop('sold', axis=1)
    
    # train XGBoost by using best_params
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_metric='rmse')
        
    # predict for test
    eval_preds[X_test.index] = model.predict(X_test)
    
    # save model for test
    modelfile = 'BestModel4test_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    
    del df_store, X_train, y_train, X_test, model
    gc.collect()
    

## Train & Predict by best model

In [ ]:
# train & predict (best model)
df_all = pd.read_pickle(os.path.join(data_dir, 'data_all.pkl'))
for store in tqdm(stores):
    # best model for valid
    xgboost_by_bestparam_for_valid(df_all, store, study.best_params, valid_preds)
    
    # best model for test
    xgboost_by_bestparam_for_test(df_all, store, study.best_params, eval_preds)


In [ ]:
del df_all
gc.collect()

# Make submission

In [ ]:
#Set actual equal to false if you want to top in the public leaderboard :P
actual = True
if actual == False:
    #Get the validation results(We already have them as less than one month left for competition to end)
    validation = sales[['id']+['d_' + str(i) for i in range(1914,1942)]]
    validation['id']=pd.read_csv(os.path.join(rawdata_dir, 'sales_train_validation.csv')).id
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
else:
    #Get the actual validation results
    valid['sold'] = valid_preds
    validation = valid[['id','d','sold']]
    validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
    validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

#Get the evaluation results
test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
#Remap the category id to their respective categories
evaluation.id = evaluation.id.map(d_id)

#Prepare the submission
submit = pd.concat([validation,evaluation]).reset_index(drop=True)
submit_file = 'xgboost_cv_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.csv'
submit.to_csv(os.path.join('submission_point', submit_file), index=False)